# Before to start read carefully the instructions provided in the README file to obtain the data and organize it.

In [ ]:
from src.utils import get_data, choose_dataset, retrieve_model, select_model_and_dataset, predict, continue_testing
from src.dataset import get_loader
import torch


buckeye_corpus_path = "../Word-Segmenter/data/Buckeye"
train_indices = "../Word-Segmenter/data/buckeye_train_data.txt"
val_indices = "../Word-Segmenter/data/buckeye_val_data.txt"
test_indices = "../Word-Segmenter/data/buckeye_test_data.txt"

timit_testset_path = "../Word-Segmenter/data/TIMIT/data/TEST"
timit_indices= "../Word-Segmenter/data/timit_test_data.txt"

ntimit_testset_path = "../Word-Segmenter/data/NTIMIT"
ntimit_indices= "../Word-Segmenter/data/ntimit_test_data.txt"

test_sets, ntimit_exists= get_data(buckeye_corpus_path, 
                                   train_indices, 
                                   val_indices, 
                                   test_indices, 
                                   timit_testset_path, 
                                   timit_indices, 
                                   ntimit_testset_path, 
                                   ntimit_indices)

# Follow the instructions displayed under the cell.

In [ ]:
continue_to_test='y'

while continue_to_test=='y':

    model_name, dataset= select_model_and_dataset(ntimit_exists)

    SR=16000
    HOP_LENGTH=int(0.025*SR)
    NUM_CLASSES=3
    BATCH_SIZE= 32
    N_DEV=0
    time=0
    frames_out=0
    verbose=False
    freeze=True
    device = torch.device(f'cuda:{N_DEV}' if torch.cuda.is_available() else 'cpu')

    wavs, labels, bounds, tolerance= choose_dataset(dataset, test_sets, SR)

    print('Wavs shape:', wavs.shape, "Labels shape:", labels.shape, "Bounds shape: ", bounds.shape, "\n")

    # Get the loader
    loader= get_loader(wavs, 
                       labels, 
                       bounds, 
                       BATCH_SIZE,
                       type='test')

    # Retrieve the time axis value of the input and the number of frames in the model's output.
    a, l, b = next(iter(loader))
    time = a.shape[1]
    frames_out = l.shape[1]

    print(f'Time points: {time}')
    print(f'Frames out: {frames_out}\n')

    # Get the model
    model= retrieve_model(model_name, 
                          time, 
                          frames_out, 
                          NUM_CLASSES, 
                          verbose, 
                          freeze)

    bounds = predict( model, 
                    loader, 
                    device, 
                    tolerance = tolerance, 
                    hop_length = HOP_LENGTH, 
                    frame_selection = "mid",
                    desc= 'Test',
                    plot_bounds=False,
                    batch_plot_id=1)
    
    continue_to_test= continue_testing()